## CONFIGURAÇÃO DO AMBIENTE E IMPORT DAS BIBLIOTECAS NECESSÁRIAS

In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

import string
import re
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dudu_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dudu_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dudu_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## DOWNLOAD DO DATASET A SER TRABALHADO

In [3]:
# IMPORTANDO O DATASET 

df_raw = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\train (2).csv")

In [4]:
# CRIANDO UM BACKUP

df = df_raw.copy()

## DATA UNDERSTANDING

In [5]:
# PRÉ-VISUALIZAÇÃO DOS DADOS

df.head(10)

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1
5,"rt @user @user não sei oq é mais chocante, um ...",1
6,@user pois é mano que coisa chata da porra,0
7,rt @user eu odeio sentir ódio de alguém pq eu ...,1
8,se eu ganhar raspo minha cabeça\n https://t.co...,0
9,rt @user @user vlw mano tmj 鉂ゐ煉煆�,0


In [6]:
# DIMENSÕES DO DATASET

print("O dataset possui",df.shape[0],"linhas e",df.shape[1],"colunas")

O dataset possui 16800 linhas e 2 colunas


In [7]:
# ANÁLISE DA PROPORÇÃO DE CADA UMA DAS CLASSES

proporcao = df['label'].value_counts().to_frame()

proporcao['%'] = (proporcao['count'] / df.shape[0]) * 100

proporcao['%'] = proporcao['%'].round()

proporcao

,count,%
label,,
0,9425,56.0
1,7375,44.0


## DATA CLEANING AND DATA PREPARATION

In [8]:
# Criação de uma coluna com os tratamentos

df['text_adjusted'] = df['text']

In [9]:
# Alterando todos os caracteres para minúsculos

df['text_adjusted'] = df['text_adjusted'].str.lower()

In [10]:
# Retirada do termo "@user" - "rt" - "\n", pois referenciam apenas o ato de retweetar, o perfil do tweet e quebras de linhas

df['text_adjusted'] = df['text_adjusted'].str.replace("@user","")

df['text_adjusted'] = df['text_adjusted'].str.replace("rt ","")

df['text_adjusted'] = df['text_adjusted'].str.replace("\n","")

In [11]:
# Retirada de todas as pontuações dos textos

for i in string.punctuation:
    df['text_adjusted'] = df['text_adjusted'].str.replace(i, "")

In [12]:
# Retirada de links encurtados, todos começam com https 

def remove_https(text):
    return re.sub(r'https\S+', '', text)

df['text_adjusted'] = df['text_adjusted'].apply(remove_https)

In [13]:
# Função para remover emojis de um texto
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # símbolos e pictogramas
                               u"\U0001F680-\U0001F6FF"  # transporte e símbolos mapas
                               u"\U0001F1E0-\U0001F1FF"  # bandeiras (iOS)
                               u"\U00002500-\U00002BEF"  # caracteres chineses comuns
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # símbolo variante de texto
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Retirando emojis do texto 
df['text_adjusted'] = df['text_adjusted'].apply(remove_emojis)

In [14]:
# Retirada de acentos 

for i,v in enumerate(df['text_adjusted']):
    df['text_adjusted'][i] = unidecode.unidecode(v)

In [15]:
# Definindo as stopwords em ptbr
stop_words = set(stopwords.words('portuguese'))  # Altere para o idioma desejado

# Retirada de stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['text_adjusted'] = df['text_adjusted'].apply(remove_stopwords)

## MODELAGEM

In [16]:
# Criando um indice para termo utilizado nos tweets

termos = {}
id = 0
for i in df['text_adjusted']:
    for v in i.split():
        termos[id] = v
        id += 1

## Treinamento SVC

In [17]:
df_treino = df[['text_adjusted','label']]

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Divisão dos dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_treino['text_adjusted'], df_treino['label'], test_size=0.2, random_state=42)

# Vetorização do texto usando TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Treinamento do modelo SVM
model = SVC(kernel='linear')
model.fit(X_train_vect, y_train)

# Fazendo previsões
predictions = model.predict(X_test_vect)

# Avaliando o modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.7404761904761905


## Treinamento SGDClassifier

In [19]:
df_treino = df[['text_adjusted','label']]

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix

# Divisão dos dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_treino['text_adjusted'], df_treino['label'], test_size=0.2, random_state=42)

# Vetorização do texto usando TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Treinamento do modelo 
clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
clf.fit(X_train_vect, y_train)

# Fazendo previsões
predictions_clf = clf.predict(X_test_vect)

# Avaliando o modelo
accuracy_clf = accuracy_score(y_test, predictions_clf)
print("Acurácia do modelo:", accuracy_clf)

Acurácia do modelo: 0.6705357142857142


## Teste SVC

In [21]:
# Teste

df_teste = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\test (4).csv")

In [22]:
# Filtrando apenas a coluna com os tweets

x_test_svc = df_teste[['text']]

In [23]:
# Criação de um pipeline de tratamento dos dados de teste (os mesmos que foram feitos para os de treino)

def pipeline(text):
    text = text.str.lower() # tudo minusculo
    
    text = text.str.replace("@user","")
    text = text.str.replace("rt ","") # sem termos desnecessários
    text = text.str.replace("\n","")
    
    for i in string.punctuation:
        text = text.str.replace(i, "") # sem pontuação
    
    text = text.apply(remove_https) # sem links
    
    text = text.apply(remove_emojis) # sem emojis
    
    for i,v in enumerate(text):
        text[i] = unidecode.unidecode(v) # normalizados (sem acento)
    
    text = text.apply(remove_stopwords) # sem stopwords

    return text

# Aplicando o pipeline na coluna de test
x_test_svc = x_test_svc.apply(pipeline)

# Transformando em array
x_test_svc = x_test_svc['text']

# Vetorizando 
x_test_svc_vect = vectorizer.transform(x_test_svc)

In [24]:
# Predição

predictions_svc = model.predict(x_test_svc_vect)

In [25]:
# Criando coluna com as predições
df_teste['label'] = predictions_svc

# Formatando para o modo de submissão
df_teste = df_teste.drop('text', axis=1)

In [26]:
df_teste.to_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\submissao_svc2.csv", index=False)

## Teste SGDClassifier

In [27]:
# Teste

df_teste = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\test (4).csv")

In [28]:
# Filtrando apenas a coluna com os tweets

x_test_clf = df_teste[['text']]

In [29]:
# Criação de um pipeline de tratamento dos dados de teste (os mesmos que foram feitos para os de treino)

def pipeline(text):
    text = text.str.lower() # tudo minusculo
    
    text = text.str.replace("@user","")
    text = text.str.replace("rt ","") # sem termos desnecessários
    text = text.str.replace("\n","")
    
    for i in string.punctuation:
        text = text.str.replace(i, "") # sem pontuação
    
    text = text.apply(remove_https) # sem links
    
    text = text.apply(remove_emojis) # sem emojis
    
    for i,v in enumerate(text):
        text[i] = unidecode.unidecode(v) # normalizados (sem acento)
    
    text = text.apply(remove_stopwords) # sem stopwords

    return text

# Aplicando o pipeline na coluna de test
x_test_clf = x_test_clf.apply(pipeline)

# Transformando em array
x_test_clf = x_test_clf['text']

# Vetorizando 
x_test_clf_vect = vectorizer.transform(x_test_clf)

In [30]:
# Predição

predictions_clf = clf.predict(x_test_clf_vect)

In [31]:
# Criando coluna com as predições
df_teste['label'] = predictions_clf

# Formatando para o modo de submissão
df_teste = df_teste.drop('text', axis=1)

In [32]:
df_teste.to_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\submissao_sgdclassifier.csv", index=False)